# TFG

In [ ]:
import pandas as pd

# import sklearn

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

# ta
# ta pandas

testing the matrix build

In [2]:
# read data
csv_hour_file = '../data/hour_14_metrics.csv'
df = pd.read_csv(csv_hour_file, parse_dates=['Datetime'])
df = df[['Datetime', 'MarginalES']]

# Date range for the training matrix
train_start_date = '2018-01-01'
train_end_date = '2022-01-01'

train_subset_df = df[(df['Datetime'] >= train_start_date) & (df['Datetime'] <= train_end_date)]



##### 

# probar con eso tambien
# error de retorno con 0s
#! sumarle 0,001 a todo para que nunca sea 0
#! limpiar numeros negativos
    # subsanado con limpieza y posterior interpolacion

# benchmark 1 precios absolutos
# benchmark 2 precios relativos
# benchmark 3 indicadores tecnicos
    # 1. media movil
    # 2. momentum
    # 3. RSI
    # 4. añdir los precios absolutos o relativos
# benchmark 4 random forest



# precios reales y precios relativos

# Sliding window size
window_size = 3

In [3]:
# Function to create sliding windows of data
def create_sliding_window(data, window_size):
    X, y = [], []  # Initialize lists for input features (X) and target values (y)
    
    for i in range(len(data) - window_size):
        # Extract a window of size 'window_size' from the data
        X.append(data.iloc[i:i+window_size, 1:].values.flatten())  
        
        # The label is the value right after the current window
        y.append(data.iloc[i + window_size, 1])  
    
    # Convert the lists to DataFrame/Series for easier use in training
    return pd.DataFrame(X), pd.Series(y)

In [4]:
# Create sliding window matrix
X_train, y_train = create_sliding_window(train_subset_df, window_size)

Print to check the matrix

In [5]:
# print the shape of the matrix
print(X_train.shape, y_train.shape)

# print the actual matrix
print(X_train.head())
print(y_train.head())

# Lasso
# .coef_ da los pesos del modelo lineal
# las que sean cero o muy bajitas no van a tener peso en la decision

(1459, 3) (1459,)
       0      1      2
0   2.30  48.02  52.00
1  48.02  52.00  48.19
2  52.00  48.19  52.02
3  48.19  52.02  52.07
4  52.02  52.07  54.07
0    48.19
1    52.02
2    52.07
3    54.07
4    69.42
dtype: float64


## Build a simple regression model and train with the matrix

In [6]:
# Build a simple regression model and train with the matrix
model_lr = LinearRegression()

# Train with the training subset
# model_lr.train(X_train, y_train)
model_lr.fit(X_train, y_train)

#! en vez de score llamar a predict y guardar los valors

#! bucle for
# predict


# Fit with the next subset up to the row before prediction

# Date range for the testing (fit) matrix
test_start_date = '2022-01-02'
test_end_date = '2025-01-01'

test_subset_df = df[(df['Datetime'] >= test_start_date) & (df['Datetime'] <= test_end_date)]

# Create sliding window matrix
X_test, y_test = create_sliding_window(test_subset_df, window_size)

#! ERROR - you cannot fit the regression again with test data - its retraining the whole thing
#! Fit the model w test data
# model_lr.fit(X_test, y_test)

# llamar a fit con toda la matriz, con la anchura ventana deslizante
# y altura num of days back

#! me falta score con test
# llama a predict con la x y compara con mis ys
# guardar errores y predicciones
# std dev
# percentil

# pickle para guardar sets de numeros en vez de numpy ??

# Predict the target values
# y_pred = model_lr.predict(X) # SOLO la lonchita
# predict o score para otros modelos
y_pred = model_lr.predict(X_test) #! SOLO la lonchita ???
mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse}")

# mirar tambien Expectation Short-Fall
# mirar cual seria la peor prediccion
# la media del 5% peor de resultados
# buscar que el caso peor no sea tan malo?

# linear regressor has mean error 0 that why we do error cuadratico o absoluto

# llamar a predict con solo la ultima fila (ancho)
X_last = X_test.iloc[-1, :].values.reshape(1, -1)
y_pred = model_lr.predict(X_last)
# mse = mean_squared_error(y_test, y_pred)
print(f"x_last: {X_last}")
print(f"y_pred: {y_pred}")

# Calculate the mean squared error and R2 score
# mse = mean_squared_error(y, y_pred)
# print('Mean squared error:', mse)
# r2 = r2_score(y, y_pred)
# print('R2 score:', r2)

# # Print 20 y and y_pred to visualize the difference
# print('Actual vs Predicted')
# print(pd.DataFrame({'Actual': y, 'Predicted': y_pred}).head(20))

# # Plot the actual vs. predicted values
# plt.figure(figsize=(10, 6))
# plt.plot(subset_df['Datetime'][window_size:], y, label='Actual')
# plt.plot(subset_df['Datetime'][window_size:], y_pred, label='Predicted')
# plt.legend()
# plt.show()

Test MSE: 1112.892958044297
x_last: [[110.01 108.88 116.02]]
y_pred: [112.48665834]


## Now with the return values

Pasos a seguir

Calcular el retorno (en vez de usar los precios)
Interpolar (rellenar valores)
Coger el set completo y partir en dos

Entrenar con la primera mitad (bloque 1 de datos)
    - Train

Coger todo el set de datos anterior a la fila que queremos (bloque 2 de datos)
    - Fit

Hacer predict de la siguiente fila al set de datos anterior (resultado a predecir)


In [7]:
window_size = 30

# read return data
csv_hour_file = '../data/clean_return_data.csv'
df = pd.read_csv(csv_hour_file, parse_dates=['Datetime'])
df = df[['Datetime', 'Return']]

# Date range for the training matrix
train_start_date = '2018-01-02'
train_end_date = '2022-01-01'

train_subset_df = df[(df['Datetime'] >= train_start_date) & (df['Datetime'] <= train_end_date)]

# Create sliding window matrix
X_train, y_train = create_sliding_window(train_subset_df, window_size)

# Build a simple regression model and train with the matrix
model_lr = LinearRegression()

# Train with the training subset
# model_lr.train(X_train, y_train)
model_lr.fit(X_train, y_train)


# Fit with the next subset up to the row before prediction

# Date range for the testing (fit) matrix
test_start_date = '2022-01-02'
test_end_date = '2025-01-01'

test_subset_df = df[(df['Datetime'] >= test_start_date) & (df['Datetime'] <= test_end_date)]

# Create sliding window matrix
X_test, y_test = create_sliding_window(test_subset_df, window_size)

y_pred = model_lr.predict(X_test) #! SOLO la lonchita ???
mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse}")

# linear regressor has mean error 0 that why we do error cuadratico o absoluto

# llamar a predict con solo la ultima fila (ancho)
X_last = X_test.iloc[-1, :].values.reshape(1, -1)
y_pred = model_lr.predict(X_last)
# mse = mean_squared_error(y_test, y_pred)
print(f"x_last: {X_last}")
print(f"y_pred: {y_pred}")

Test MSE: 192605.69284437993
x_last: [[ 2.17391304e-01  1.60714286e-01  3.53846154e-03 -1.68250805e-01
  -8.75495346e-03 -6.37783563e-02 -9.67030785e-01 -8.67469880e-01
   7.75000000e+00  2.89246753e+01  5.54639354e-02  1.43092105e-01
  -8.92086331e-03 -2.16390825e-01 -2.97637795e-01  2.71828014e-01
   1.03702167e-01 -1.50051677e-01 -2.21755472e-01  2.18892045e-01
  -5.83032281e-01 -5.70150922e-01 -7.84135241e-01  9.77710843e+00
   1.43991056e+00  2.06185567e-01  2.50712251e-02  1.91773207e-02
  -1.02717935e-02  6.55767818e-02]]
y_pred: [0.6227191]


In [8]:
# For later on...
#! Build a simple random forest model and train with the matrix

# model_rf = sklearn RandomForest()
# model_rf.fit(X, y)

# # llamar a fit con toda la matriz, con la anchura ventana deslizante
# # y altura num of days back

# # Predict the target values
# y_pred = model_rf.predict(X)

# # llamar a predict con solo una fila (ancho)

# # Calculate the mean squared error and R2 score
# mse = mean_squared_error(y, y_pred)
# print('Mean squared error:', mse)
# r2 = r2_score(y, y_pred)
# print('R2 score:', r2)

# # Print 20 y and y_pred to visualize the difference
# print('Actual vs Predicted')
# print(pd.DataFrame({'Actual': y, 'Predicted': y_pred}).head(20))

# # Plot the actual vs. predicted values
# plt.figure(figsize=(10, 6))
# plt.plot(subset_df['Datetime'][window_size:], y, label='Actual')
# plt.plot(subset_df['Datetime'][window_size:], y_pred, label='Predicted')
# plt.legend()
# plt.show()

## Añadiendo columnas
Calcularemos varias metricas con la libreria "ta" - technical analysis

### Próximos pasos
- Añadir columnas de métricas de precios
  - Media móvil corta (a pocos días)
  - Media móvil larga (a muchos días)
  - RSI
  - Volatilidad
  - Momentum
- Probar regresión otra vez
- Verificar con
  - MSE
  - RMSE
  - 95th percentil
  - R² modificado?
- Mirar foto - usar en el sumatorio de abajo, las etiquetas de train
- Probar Lasso
- Mirar error (distintas métricas)
- Mirar pesos que escoge - aprender del modelo


In [ ]:
# Añadir columnas de metricas de precios
    # media movil corta (a pocos dias)
    # media movil larga (a muchos dias)
    # RSI
    # volatilidad
    # momentum

# probar regresion otra vez
    # verificar con
    # mse
    # rmse
    # 95th percentil

# R^2 modificado?
# mirar foto - usar en el sumatorio de abajo, las etiquetas de train

window_size = 30

# read return data
csv_hour_file = '../data/ta_metrics/new_price_metrics_hour_14.csv'
# csv_hour_file = '../data/ta_metrics/new_return_metrics_hour_14.csv'

df = pd.read_csv(csv_hour_file, parse_dates=['Datetime'])

#! Expandir esto para incluir nuevas metricas (mas columnas)
# df = df[['Datetime', 'MarginalES']]
# df = df[['Datetime', 'Return']] #! esto no vale

# Select all relevant columns except 'Datetime'
feature_columns = df.columns[1:]  # Exclude 'Datetime'
df = df[['Datetime'] + list(feature_columns)]

# Function to create sliding windows of data
def create_sliding_window_more_columns(data, window_size):
    X, y = [], []  # Initialize lists for input features (X) and target values (y)

    #! REWORK THIS CODE - matrix shall have more columns now    
    for i in range(len(data) - window_size):
        # Extract a window of 'window_size' rows for all feature columns (except 'Datetime')
        X.append(data.iloc[i:i+window_size, 1:].values.flatten())
        y.append(data.iloc[i + window_size]['MarginalES'])
        # The label is the 'Return' value right after the current window
        # y.append(data.iloc[i + window_size]['Return'])
    
    # Convert the lists to DataFrame/Series for easier use in training
    return pd.DataFrame(X), pd.Series(y)

# Date range for the training matrix
train_start_date = '2018-08-01' # less dates because of incomplete data when doing rolling windows
train_end_date = '2022-08-01'

train_subset_df = df[(df['Datetime'] >= train_start_date) & (df['Datetime'] <= train_end_date)]

# Create sliding window matrix
#! modify to include more columns ????
X_train, y_train = create_sliding_window_more_columns(train_subset_df, window_size)

# Build a simple regression model and train with the matrix
model_lr = LinearRegression()

# Train with the training subset
# model_lr.train(X_train, y_train)
model_lr.fit(X_train, y_train)


# Fit with the next subset up to the row before prediction

# Date range for the testing (fit) matrix
test_start_date = '2022-08-02'
test_end_date = '2025-01-01'

test_subset_df = df[(df['Datetime'] >= test_start_date) & (df['Datetime'] <= test_end_date)]

# Create sliding window matrix
X_test, y_test = create_sliding_window_more_columns(test_subset_df, window_size)

y_pred = model_lr.predict(X_test) #! SOLO la lonchita ???
mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse}")

# linear regressor has mean error 0 that why we do error cuadratico o absoluto

# llamar a predict con solo la ultima fila (ancho)
X_last = X_test.iloc[-1, :].values.reshape(1, -1)
y_pred = model_lr.predict(X_last)
# mse = mean_squared_error(y_test, y_pred)
print(f"x_last: {X_last}")
print(f"y_pred: {y_pred}")

Test MSE: 1311128222.606367
x_last: [[ 1.12000000e+02  1.04066667e+02  1.04192000e+02  1.04520000e+02
   8.68930000e+01  8.08030000e+01  6.56222000e+01  5.98496667e+01
   5.25071000e+01  4.33714000e+01  9.17530722e+01  8.19161976e+01
   3.22580645e+00  6.15928437e+01  5.80593008e+01  5.62017735e+01
   5.40078995e+01  1.30000000e+02  1.11333333e+02  1.09078000e+02
   1.12808571e+02  9.01910000e+01  8.28083333e+01  6.82206000e+01
   6.14885000e+01  5.38071000e+01  4.40066500e+01  9.76372150e+01
   8.54779608e+01  1.58964143e+02  1.62400000e+05  6.42171344e+01
   6.03785812e+01  5.60752169e+01  1.30460000e+02  1.24153333e+02
   1.14532000e+02  1.11631429e+02  9.95420000e+01  8.63313333e+01
   6.92614000e+01  6.26745000e+01  5.51118000e+01  4.45570000e+01
   1.02686874e+02  8.88099637e+01  2.92951807e+02  6.63606223e+01
   6.43841783e+01  6.04909167e+01  5.61294877e+01  1.08510000e+02
   1.22990000e+02  1.14594000e+02  1.12051429e+02  1.10061000e+02
   8.69450000e+01  6.98332000e+01  6.448

## probar lasso
- mirar error (distintas metricas)
- mirar pesos que escoge - aprender del modelo